## GOLD TO API LAYER
### API Layer


In [1]:
import json
import pandas as pd
from ETLTools import GlobalPath, utils

In [2]:
# Define file paths using GlobalPath
current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holdings/CurrentHoldings_data.csv"
)
stockprice_silver_file_path = GlobalPath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)
holdingshistory_gold_file_path = GlobalPath(
    "DATA/GOLD/Holdings/HoldingsHistory_data.csv"
)
profitloss_gold_file_path = GlobalPath(
    "DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)
api_file_path = GlobalPath("DATA/API/API_data.json")

In [3]:
def group_holdings_data(df_holding):
    """Groups holding data by scrip_name, symbol, exchange, and segment to calculate totals and min datetime."""
    df_holding["datetime"] = pd.to_datetime(df_holding["datetime"])
    df_current_holding = (
        df_holding.groupby(["scrip_name", "symbol", "exchange", "segment"])
        .agg(
            total_quantity=("quantity", "sum"),
            total_amount=("amount", "sum"),
            min_datetime=("datetime", "min"),
        )
        .reset_index()
    )
    df_current_holding["avg_price"] = (
        df_current_holding["total_amount"]
        / df_current_holding["total_quantity"]
    )
    return df_current_holding

In [4]:
def process_stock_prices(df_stockprice):
    """Processes stock prices to extract the latest closing price for each symbol."""
    df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])
    df_stockprice["close_price"] = df_stockprice["close"]
    idx = df_stockprice.groupby("symbol")["date"].idxmax()
    return df_stockprice.loc[idx].reset_index(drop=True)

In [5]:
def merge_and_calculate_pnl(df_current_holding, df_stockprice):
    """Merges holdings data with stock prices and calculates PnL."""
    df_current_holding = pd.merge(
        df_current_holding,
        df_stockprice[["symbol", "close_price"]],
        on="symbol",
        how="left",
    )
    df_current_holding["close_amount"] = (
        df_current_holding["close_price"] * df_current_holding["total_quantity"]
    )
    df_current_holding["pnl_amount"] = (
        df_current_holding["close_amount"] - df_current_holding["total_amount"]
    )
    return df_current_holding.round(2)

In [6]:
def prepare_current_holding_data(df_current_holding, df_holding):
    """Prepares the current holding data with historical records."""
    current_holding = []
    for _, row in df_current_holding.iterrows():
        df_filtered = df_holding[
            (df_holding["scrip_name"] == row["scrip_name"])
            & (df_holding["symbol"] == row["symbol"])
            & (df_holding["exchange"] == row["exchange"])
            & (df_holding["segment"] == row["segment"])
        ]
        row = row.to_dict()
        row["history"] = df_filtered.to_dict(orient="records")
        current_holding.append(row)
    return current_holding

In [7]:
def process_holdings_history(df_holdings):
    """Processes holdings history data to calculate trends."""
    df_holdings["date"] = pd.to_datetime(df_holdings["date"]).dt.date
    df_holdings_trands = (
        df_holdings.groupby("date")[
            [
                "holding_amount",
                "open_amount",
                "high_amount",
                "low_amount",
                "close_amount",
            ]
        ]
        .sum()
        .reset_index()
    )
    df_holdings_trands = df_holdings_trands.round(2).rename(
        columns={
            col: col.replace("_amount", "")
            for col in df_holdings_trands.columns
        }
    )
    return (
        df_holdings_trands[["date", "open", "high", "low", "close", "holding"]]
        .sort_values(by=["date"])
        .reset_index(drop=True)
    )

In [8]:
def process_profit_loss_data(df_profitloss):
    """Processes profit and loss data to calculate days and grouping by segment, exchange, symbol, and stock_name."""
    df_profitloss["stock_name"] = df_profitloss.apply(
        lambda row: (
            row["symbol"] if row["symbol"] == "NIFTY" else row["scrip_name"]
        ),
        axis=1,
    )
    df_profitloss["open_datetime"] = pd.to_datetime(
        df_profitloss["open_datetime"]
    )
    df_profitloss["close_datetime"] = pd.to_datetime(
        df_profitloss["close_datetime"]
    )
    df_profitloss["days"] = (
        df_profitloss["close_datetime"] - df_profitloss["open_datetime"]
    ).dt.days
    df_profitloss = df_profitloss.sort_values(
        by=["segment", "symbol"]
    ).reset_index(drop=True)

    profit_loss_data = []
    grouped = df_profitloss.groupby(
        ["segment", "exchange", "symbol", "stock_name"]
    )

    for (segment, exchange, symbol, stock_name), group in grouped:
        group_dict = {
            "segment": segment,
            "exchange": exchange,
            "symbol": symbol,
            "stock_name": stock_name,
            "days": (
                group["close_datetime"].max() - group["open_datetime"].min()
            ).days,
            "quantity": group["quantity"].sum(),
            "avg_price": round(
                group["open_amount"].sum() / group["quantity"].sum(), 2
            ),
            "sell_price": round(
                group["close_amount"].sum() / group["quantity"].sum(), 2
            ),
            "pnl": group["pnl_amount"].sum(),
            "history": group[
                [
                    "scrip_name",
                    "position",
                    "quantity",
                    "days",
                    "open_datetime",
                    "open_price",
                    "open_amount",
                    "close_datetime",
                    "close_price",
                    "close_amount",
                    "pnl_amount",
                    "pnl_percentage",
                ]
            ].to_dict(orient="records"),
        }
        profit_loss_data.append(group_dict)
    return profit_loss_data

In [9]:
# Main flow
df_holding = utils.read_data(current_holding_records_file_path.path)
df_current_holding = group_holdings_data(df_holding)
df_stockprice = utils.read_data(stockprice_silver_file_path.path)
df_stockprice = process_stock_prices(df_stockprice)
df_current_holding = merge_and_calculate_pnl(df_current_holding, df_stockprice)
current_holding = prepare_current_holding_data(df_current_holding, df_holding)

df_holdings = utils.read_data(holdingshistory_gold_file_path.path)
holdings_trands = process_holdings_history(df_holdings).to_dict(
    orient="records"
)

df_profitloss = utils.read_data(profitloss_gold_file_path.path)
profit_loss_data = process_profit_loss_data(df_profitloss)

Reading data from : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Holdings\CurrentHoldings_data.csv
Reading data from : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SILVER\StockPrice\StockPrice_data.csv
Reading data from : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Holdings\HoldingsHistory_data.csv
Reading data from : C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv


In [10]:
# Create the final output dictionary for the API
output = {
    "financial_summary": {
        "invested_value": round(df_current_holding["total_amount"].sum(), 2),
        "current_value": round(df_current_holding["close_amount"].sum(), 2),
        "pnl_value": round(df_current_holding["pnl_amount"].sum(), 2),
    },
    "current_holding_data": current_holding,
    "holdings_trands_data": holdings_trands,
    "profitloss_summary": {
        "invested_value": round(df_profitloss["open_amount"].sum(), 2),
        "sold_value": round(df_profitloss["close_amount"].sum(), 2),
        "pnl_value": round(df_profitloss["pnl_amount"].sum(), 2),
    },
    "profit_loss_data": profit_loss_data,
}

# Write the result to a JSON file
with open(api_file_path.path, "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, indent=4, default=str)

print(f"Data written to {api_file_path}")

Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\API\API_data.json
